In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install scala


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement scala (from versions: none)
ERROR: No matching distribution found for scala


In [3]:
from pyspark.sql import SparkSession
import logging 
import os
from pyspark.sql.types import StructType
from pyspark.sql import DataFrame
from pyspark.sql.functions import col,lit,when,avg,count,sum
from pyspark.sql import functions as F
from collections import Counter
import os
os.environ['JAVA_HOME'] = 'C:\\java'
import os
os.environ['HADOOP_HOME'] = 'C:\\hadoop\\hadoop-3.4.1'
import warnings
warnings.filterwarnings('ignore')
# Set environment variablesp3
os.environ['PATH'] += os.pathsep +"C:\\hadoop\\hadoop-3.4.1\\bin"
os.environ['HIVE_HOME'] = 'C:\\hive'  
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HIVE_HOME'], 'bin')
os.environ['CLASSPATH'] = os.pathsep.join([os.environ.get('HIVE_HOME') + '\\lib\\*', os.environ.get('HADOOP_HOME') + '\\share\\hadoop\\common\\*'])


In [4]:
import os
import logging
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

def create_spark_session(app_name="Data Warehouse Project", warehouse_dir=None, enable_hive_support=False, hive_metastore_uri=None):
    # Initialize logger
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(app_name)
    
    try:
        logger.info("Initializing Spark Session")

        # Initialize SparkSession builder
        builder = SparkSession.builder.appName(app_name)

        # Set warehouse directory
        if warehouse_dir is None:
            warehouse_dir = os.path.join(os.getcwd(), "spark-warehouse")
        builder.config("spark.sql.warehouse.dir", warehouse_dir)

        # Add additional configurations
        builder.config("spark.executor.memory", "4g")
        builder.config("spark.driver.memory", "16g")
        builder.config("spark.driver.extraJavaOptions", "-XX:+UseCompressedOops")
        builder.config("spark.sql.shuffle.partitions", "200")

        # Set environment variables for Java, Hadoop, and Hive
        os.environ['JAVA_HOME'] = 'C:\\java'  # Update with your Java installation path
        os.environ['HADOOP_HOME'] = 'C:\\hadoop\\hadoop-3.4.1'  # Update with your Hadoop installation path
        os.environ['HIVE_HOME'] = 'C:\\hive\\apache-hive-3.1.2-bin'  # Update with your Hive installation path
        os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HADOOP_HOME'], 'bin')
        os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HIVE_HOME'], 'bin')
        os.environ['CLASSPATH'] = os.pathsep.join([os.environ.get('HIVE_HOME') + '\\lib\\*', os.environ.get('HADOOP_HOME') + '\\share\\hadoop\\common\\*'])

        # Enable Hive support if requested
        if enable_hive_support:
            if hive_metastore_uri:
                builder.config("hive.metastore.uris", hive_metastore_uri)  # Remote Metastore URI
            else:
                logger.warning("Hive support enabled, but no metastore URI provided. Using default Hive configuration.")
            builder.enableHiveSupport()

        # Create SparkSession
        spark = builder.getOrCreate()

        logger.info("Spark Session Created Successfully")
        logger.info(f"Spark SQL Warehouse Directory: {warehouse_dir}")
        if enable_hive_support:
            logger.info("Hive support is enabled.")

        return spark

    except AnalysisException as ae:
        logger.error(f"AnalysisException: {str(ae)}")
        raise
    except Exception as e:
        logger.error(f"An error occurred while creating the Spark session: {str(e)}")
        raise

# Example usage of the session creation:
try:
    spark = create_spark_session(
        app_name="My Data Warehouse", 
        enable_hive_support=True, 
        hive_metastore_uri="thrift://localhost:9083"  # Use your actual metastore URI here
    )
    # Now you can run your analytics queries
except Exception as e:
    logging.error(f"Failed to initialize Spark Session: {str(e)}")



2025-01-07 16:26:42,945 - INFO - Initializing Spark Session
2025-01-07 16:26:50,424 - INFO - Spark Session Created Successfully
2025-01-07 16:26:50,427 - INFO - Spark SQL Warehouse Directory: C:\Users\Akshata Pandit\spark-warehouse
2025-01-07 16:26:50,429 - INFO - Hive support is enabled.


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when
import logging
import smtplib
from email.message import EmailMessage

# Logging Configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("SparkJobMonitor")

# Email Alert Function
def send_email_alert(subject, body):
    sender_email = "akshatapandit7@gmail.com"
    receiver_email = "akshata.codes2332@gmail.com"
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_password = "your_email_password"

    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg.set_content(body)

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, smtp_password)
            server.send_message(msg)
        logger.info("Email alert sent successfully.")
    except Exception as e:
        logger.error(f"Failed to send email alert: {e}")

# Spark Job Execution and Monitoring
def run_spark_job():
    spark = SparkSession.builder \
        .appName("AlternativeSparkMonitoring") \
        .getOrCreate()

    try:
        # Replace with your data path
        data_path = "C:\\excels\\train.csv"
        df = spark.read.csv(data_path, header=True)

        # Perform simple data validation
        total_rows = df.count()
        null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()

        logger.info(f"Total rows processed: {total_rows}")
        logger.info(f"Null counts: {null_counts}")

        # Alert if row count is below a threshold
        if total_rows < 1000:
            alert_message = f"ALERT: Row count below threshold! Processed rows: {total_rows}"
            logger.warning(alert_message)
            send_email_alert("Row Count Alert", alert_message)

        # Alert if null values exceed a threshold
        for column, null_count in zip(df.columns, null_counts[0]):
            if null_count > 0:
                alert_message = f"ALERT: Null values detected in column '{column}': {null_count}"
                logger.warning(alert_message)
                send_email_alert("Data Quality Alert", alert_message)

    except Exception as e:
        logger.error(f"Spark job encountered an error: {e}")
        send_email_alert("Job Failure Alert", f"Spark job failed with error: {e}")

    finally:
        spark.stop()
        logger.info("Spark session stopped.")

# Run the Spark job
if __name__ == "__main__":
    run_spark_job()


2025-01-07 16:27:15,738 - INFO - Total rows processed: 1058
2025-01-07 16:27:15,741 - INFO - Null counts: [Row(Age=0, Attrition=0, BusinessTravel=0, DailyRate=0, Department=0, DistanceFromHome=0, Education=0, EducationField=0, EmployeeCount=0, EmployeeNumber=0, EnvironmentSatisfaction=0, Gender=0, HourlyRate=0, JobInvolvement=0, JobLevel=0, JobRole=0, JobSatisfaction=0, MaritalStatus=0, MonthlyIncome=0, MonthlyRate=0, NumCompaniesWorked=0, Over18=0, OverTime=0, PercentSalaryHike=0, PerformanceRating=0, RelationshipSatisfaction=0, StandardHours=0, StockOptionLevel=0, TotalWorkingYears=0, TrainingTimesLastYear=0, WorkLifeBalance=0, YearsAtCompany=0, YearsInCurrentRole=0, YearsSinceLastPromotion=0, YearsWithCurrManager=0)]
2025-01-07 16:27:16,410 - INFO - Spark session stopped.


In [6]:
warehouse_directory = " C:\\BigDataOutput"  
spark = create_spark_session(warehouse_dir=warehouse_directory, enable_hive_support=True) 

2025-01-07 16:27:16,431 - INFO - Initializing Spark Session
2025-01-07 16:27:16,434 - WARNING - Hive support enabled, but no metastore URI provided. Using default Hive configuration.
2025-01-07 16:27:16,976 - INFO - Spark Session Created Successfully
2025-01-07 16:27:16,978 - INFO - Spark SQL Warehouse Directory:  C:\BigDataOutput
2025-01-07 16:27:16,980 - INFO - Hive support is enabled.


In [7]:
def ingest_data(
    spark:SparkSession,
    file_path:str,
    schema:StructType=None,
    file_format:str="csv",
options:dict=None) -> DataFrame:
    logger = logging.getLogger("Data Ingestion") 
    logger.info(f"Ingesting data from {file_path} as {file_format}") 
    if options is None:
        options = {}

    try: 
        if file_format.lower() == "csv": 

            df = spark.read.csv(file_path, schema=schema, **options) 

        elif file_format.lower() == "json": 

            df = spark.read.json(file_path, schema=schema, **options) 

        elif file_format.lower() == "parquet": 

            df = spark.read.parquet(file_path, **options) 
        else: 
            raise ValueError(f"Unsupported file format: {file_format}") 

        logger.info(f"Data ingestion successful: {file_path}") 
        return df 
    except Exception as e: 
        logger.error(f"Error during data ingestion: {e}") 
        raise 
csv_options = { 

    "header": "true", 

    "inferSchema": "true" 

} 
data_path = "C:\\excels\\train.csv"


data_frame = ingest_data(spark, file_path=data_path, file_format="csv", options=csv_options) 

data_frame.show() 

 

2025-01-07 16:27:17,015 - INFO - Ingesting data from C:\excels\train.csv as csv
2025-01-07 16:27:18,470 - INFO - Data ingestion successful: C:\excels\train.csv


+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalanc

In [8]:
def clean_data(df:DataFrame,columns_to_drop:list=None,default_values:dict=None)-> DataFrame:
        if columns_to_drop:
            df=df.drop(*columns_to_drop)
        if default_values:
            for col_name,default_val in default_values.items():
                df=df.withColumn(col_name,when(col(col_name).isNull(),lit(default_val)).otherwise(col(col_name)))
        return df

In [9]:
from pyspark.sql import functions as F

def aggregate_data(df: DataFrame, group_by_cols: list, agg_funcs: dict) -> DataFrame:
    
    agg_exprs = [func(F.col(col_name)).alias(f"{col_name}_{func.__name__}") for col_name, func in agg_funcs.items()]
   
    return df.groupBy(*group_by_cols).agg(*agg_exprs)


In [10]:
def join_data(df1:DataFrame,df2:DataFrame,join_col:str,join_type:str="inner")->DataFrame:
    common_columns = [col for col in df1.columns if col in df2.columns and col != join_col]
    
    
    df2_renamed = df2.select([F.col(c).alias(f"{c}_df2") if c in common_columns else F.col(c) for c in df2.columns])
    
    
    joined_df = df1.join(df2_renamed, on=join_col, how=join_type)
    
  
    for col in common_columns:
        joined_df = joined_df.withColumn(col, F.coalesce(F.col(col), F.col(f"{col}_df2")))
        joined_df = joined_df.drop(f"{col}_df2")  # Drop the renamed df2 columns
    
    return joined_df
    

In [11]:
cleaned_data=clean_data(data_frame,columns_to_drop=["id"],
                        default_values=[])

In [12]:
data_frame.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Attrition: integer (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- MonthlyRate: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- Over18: string (nullable = true)
 |-- OverTime: string

In [13]:
data_frame

DataFrame[Age: int, Attrition: int, BusinessTravel: string, DailyRate: int, Department: string, DistanceFromHome: int, Education: int, EducationField: string, EmployeeCount: int, EmployeeNumber: int, EnvironmentSatisfaction: int, Gender: string, HourlyRate: int, JobInvolvement: int, JobLevel: int, JobRole: string, JobSatisfaction: int, MaritalStatus: string, MonthlyIncome: int, MonthlyRate: int, NumCompaniesWorked: int, Over18: string, OverTime: string, PercentSalaryHike: int, PerformanceRating: int, RelationshipSatisfaction: int, StandardHours: int, StockOptionLevel: int, TotalWorkingYears: int, TrainingTimesLastYear: int, WorkLifeBalance: int, YearsAtCompany: int, YearsInCurrentRole: int, YearsSinceLastPromotion: int, YearsWithCurrManager: int]

In [14]:
data_frame2=ingest_data(spark,file_path="C:\\excels\\test.csv",file_format='csv',options=csv_options)
data_frame2.show()

2025-01-07 16:27:19,974 - INFO - Ingesting data from C:\excels\test.csv as csv
2025-01-07 16:27:21,644 - INFO - Data ingestion successful: C:\excels\test.csv


+---+-----------------+---------+--------------------+----------------+---------+----------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|  EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalance|YearsAtCompany

In [15]:
cleaned_data2=clean_data(data_frame2,columns_to_drop=["id"],
                        default_values=[])

In [16]:
data_frame2.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- MonthlyRate: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- Over18: string (nullable = true)
 |-- OverTime: string (nullable = true)
 |-- PercentSalaryHike:

In [17]:
aggregated_data=aggregate_data(cleaned_data,group_by_cols=['MonthlyIncome'],agg_funcs={"MonthlyIncome":avg,"JobSatisfaction":count})

In [18]:
aggregated_data2=aggregate_data(cleaned_data,group_by_cols=["MonthlyIncome"],agg_funcs={"MonthlyIncome":avg,"JobSatisfaction":count})

In [19]:
joined_data=join_data(data_frame,data_frame,'Department')

In [20]:
joined_data

DataFrame[Department: string, Age: int, Attrition: int, BusinessTravel: string, DailyRate: int, DistanceFromHome: int, Education: int, EducationField: string, EmployeeCount: int, EmployeeNumber: int, EnvironmentSatisfaction: int, Gender: string, HourlyRate: int, JobInvolvement: int, JobLevel: int, JobRole: string, JobSatisfaction: int, MaritalStatus: string, MonthlyIncome: int, MonthlyRate: int, NumCompaniesWorked: int, Over18: string, OverTime: string, PercentSalaryHike: int, PerformanceRating: int, RelationshipSatisfaction: int, StandardHours: int, StockOptionLevel: int, TotalWorkingYears: int, TrainingTimesLastYear: int, WorkLifeBalance: int, YearsAtCompany: int, YearsInCurrentRole: int, YearsSinceLastPromotion: int, YearsWithCurrManager: int]

In [21]:
joined_data=clean_data(joined_data,columns_to_drop=["Age"],
                        default_values=[])

In [22]:
joined_data.show()

+----------+---------+--------------+---------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+---------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Department|Attrition|BusinessTravel|DailyRate|DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|        JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalance|YearsAtCompany|YearsInCurrentRole|YearsSin

In [23]:
from pyspark.sql import SparkSession
import pandas as pd

def save_data(df, output_path, file_format='csv', mode='overwrite'):
    if file_format == 'csv':
        
        if mode == 'overwrite':
            df.to_csv(output_path, index=False, mode='w')  # 'w' mode to overwrite
        elif mode == 'append':
            df.to_csv(output_path, index=False, mode='a', header=False)  # 'a' mode for appending

    elif file_format == 'json':
        
        if mode == 'overwrite':
            df.to_json(output_path, index=False, mode='w')  # 'w' mode to overwrite
        elif mode == 'append':
            df.to_json(output_path, index=False, mode='a')  # Append is not supported directly for Excel in Pandas

    elif file_format == 'parquet':
        # For Parquet, use the Pandas to_parquet method
        if mode == 'overwrite':
            df.to_parquet(output_path, index=False)
        elif mode == 'append':
            print("Append mode is not supported for Parquet format in Pandas.")
    
    else:
        print(f"Unsupported file format: {file_format}")
    
    print(f"Data saved successfully to {output_path}")

    




2025-01-07 16:27:30,174 - INFO - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-01-07 16:27:30,178 - INFO - NumExpr defaulting to 8 threads.


In [24]:
print(type(joined_data))


<class 'pyspark.sql.dataframe.DataFrame'>


In [25]:
print(type(joined_data))


<class 'pyspark.sql.dataframe.DataFrame'>


In [26]:
import logging
logging.basicConfig(level=logging.DEBUG)

joined_data_panda = joined_data.toPandas()


output_path = "C:\\jaiganesha\\output.csv"  
save_data(joined_data_panda, output_path, file_format='csv', mode='overwrite')
output_path1= "C:\\jaiganesha\\output.json"  
save_data(joined_data_panda, output_path1, file_format='json', mode='overwrite')
output_path = "C:\\jaiganesha\\output.parquet"  
save_data(joined_data_panda, output_path, file_format='parquet', mode='overwrite')



Data saved successfully to C:\jaiganesha\output.csv
Data saved successfully to C:\jaiganesha\output.json
Data saved successfully to C:\jaiganesha\output.parquet


In [27]:
def run_analytics_queries(spark: SparkSession):
    
    df_1 = spark.read.csv("C:\\jaiganesha\\output.csv", header=True, inferSchema=True)
    df_1.createOrReplaceTempView("output_directory")
    
   
    schema = ["JobRole", "metric_column"]
    data = [
        ("Sales Executive", "metric1"),
        ("Research Scientist", "metric2"),
        ("Laboratory Technician", "metric3")
    ]
    df_2 = spark.createDataFrame(data, schema)
    df_2.createOrReplaceTempView("another_directory")

  
    query_1 = """
    SELECT  
        Gender,  
        COUNT(*) AS record_count  
    FROM output_directory 
    GROUP BY Gender 
    ORDER BY record_count DESC
    """ 
    result_1 = spark.sql(query_1)
    print("Query 1 Results:")
    result_1.show()

 

  
    result_1_pandas = result_1.toPandas()
   

    
    output_path_1 = "C:\\jaiganesha\\output_query_1.csv"
    output_path_2 = "C:\\jaiganesha\\output_query_1_p.parquet"
    output_path_3 = "C:\\jaiganesha\\output_query_1_j.json"
    
    
    save_data(result_1_pandas, output_path_1, file_format='csv', mode='overwrite')
    save_data(result_1_pandas, output_path_2, file_format='parquet', mode='overwrite')
    save_data(result_1_pandas, output_path_3, file_format='json', mode='overwrite')
    
    
    
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("My Data Warehouse Without Hive") \
    .config("spark.sql.warehouse.dir", "C:\\jaiganesha\\spark-warehouse") \
    .getOrCreate()

run_analytics_queries(spark)

Query 1 Results:
+------+------------+
|Gender|record_count|
+------+------------+
|  Male|      348622|
|Female|      245984|
+------+------------+

Data saved successfully to C:\jaiganesha\output_query_1.csv
Data saved successfully to C:\jaiganesha\output_query_1_p.parquet
Data saved successfully to C:\jaiganesha\output_query_1_j.json


In [28]:
def run_analytics_queries(spark: SparkSession):
    
    df_1 = spark.read.csv("C:\\jaiganesha\\output.csv", header=True, inferSchema=True)
    df_1.createOrReplaceTempView("output_directory")

   
    query = """
    SELECT
        JobRole,
        Gender,
        AVG(DailyRate) AS avg_daily_rate
    FROM output_directory
    GROUP BY JobRole, Gender
    ORDER BY avg_daily_rate DESC
    LIMIT 10
    """
    try:
       
        result = spark.sql(query)
        print("Query Results:")
        result.show()

        result_pandas = result.toPandas()
        output_path = "C:\\jaiganesha\\output_query_alternative.csv"
        result_pandas.to_csv(output_path, index=False)
        print(f"Query results saved to: {output_path}")
        output_path1 = "C:\\jaiganesha\\output_query_alternative_p.parquet"
        result_pandas.to_csv(output_path1, index=False)
        print(f"Query results saved to: {output_path1}")
        output_path2="C:\\jaiganesha\\output_query_alternative_j.json"
        print(f"Query results saved to: {output_path2}")
        result_pandas.to_json(output_path, index=False)

    except Exception as e:
        print(f"An error occurred while executing the query: {e}")


from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("My Data Warehouse Without Hive") \
    .config("spark.sql.warehouse.dir", "C:\\jaiganesha\\spark-warehouse") \
    .getOrCreate()


run_analytics_queries(spark)


Query Results:
+--------------------+------+-----------------+
|             JobRole|Gender|   avg_daily_rate|
+--------------------+------+-----------------+
|     Human Resources|Female|            989.0|
|Healthcare Repres...|  Male|935.9795918367347|
|             Manager|Female| 931.410907913902|
|Manufacturing Dir...|  Male|862.4909090909091|
|Laboratory Techni...|Female|       849.328125|
|     Sales Executive|Female|836.8958333333334|
|   Research Director|  Male|836.3243243243244|
|Sales Representative|Female|816.8518518518518|
|Healthcare Repres...|Female|816.6153846153846|
|  Research Scientist|  Male|806.4444444444445|
+--------------------+------+-----------------+

Query results saved to: C:\jaiganesha\output_query_alternative.csv
Query results saved to: C:\jaiganesha\output_query_alternative_p.parquet
Query results saved to: C:\jaiganesha\output_query_alternative_j.json


In [29]:
def run_additional_queries(spark: SparkSession):
    # Load the dataset
    df_1 = spark.read.csv("C:\\jaiganesha\\output.csv", header=True, inferSchema=True)
    df_1.createOrReplaceTempView("output_directory")

    # List of queries
    queries = {
        "Query 1: Top Departments by Total Salary": """
        SELECT Department, SUM(DailyRate) AS total_salary 
        FROM output_directory 
        GROUP BY Department 
        ORDER BY total_salary DESC 
        LIMIT 5
        """
        ,
        "Query 2: Gender Distribution by JobRole": """
        SELECT JobRole, Gender, COUNT(*) AS gender_count 
        FROM output_directory 
        GROUP BY JobRole, Gender 
        ORDER BY JobRole, Gender
        """
    }

    
    for title, query in queries.items():
        try:
            print(f"Executing: {title}")
            result = spark.sql(query)
            result.show()

            output_path = f"C:\\jaiganesha\\{title.replace(':', '').replace(' ', '_')}.csv"
            result.toPandas().to_csv(output_path, index=False)
            print(f"Results saved to: {output_path}\n")
            output_path1= f"C:\\jaiganesha\\{title.replace(':', '').replace(' ', '_')}.parquet"
            result.toPandas().to_parquet(output_path1, index=False)
            print(f"Results saved to: {output_path1}\n")
            output_path2 = f"C:\\jaiganesha\\{title.replace(':', '').replace(' ', '_')}.json"
            result.toPandas().to_json(output_path2, index=False)

        except Exception as e:
            print(f"An error occurred while executing {title}: {e}")


run_additional_queries(spark)


Executing: Query 1: Top Departments by Total Salary
+--------------------+------------+
|          Department|total_salary|
+--------------------+------------+
|Research & Develo...|   403068691|
|               Sales|    80312397|
|     Human Resources|     1130196|
+--------------------+------------+

Results saved to: C:\jaiganesha\Query_1_Top_Departments_by_Total_Salary.csv

Results saved to: C:\jaiganesha\Query_1_Top_Departments_by_Total_Salary.parquet

Executing: Query 2: Gender Distribution by JobRole
+--------------------+------+------------+
|             JobRole|Gender|gender_count|
+--------------------+------+------------+
|Healthcare Repres...|Female|       27339|
|Healthcare Repres...|  Male|       34349|
|     Human Resources|Female|         342|
|     Human Resources|  Male|         798|
|Laboratory Techni...|Female|       44864|
|Laboratory Techni...|  Male|       86924|
|             Manager|Female|       18537|
|             Manager|  Male|       20460|
|Manufacturin

In [30]:
query_1 = """
SELECT 
    Department, 
    EducationField, 
    AVG(MonthlyIncome) AS avg_monthly_income 
FROM output_directory 
GROUP BY Department, EducationField 
ORDER BY avg_monthly_income DESC
"""
result_1 = spark.sql(query_1)
print("Query 1: Average Monthly Income by Department and Education Field")
result_1.show()

result1_p=result_1.toPandas()
output_path= "C:\\jaiganesha\\avaragemonthlyincome.csv"
result1_p.to_csv(output_path,index=False)
print(f"the output saved successfully to the path{output_path}\n")
output_path1= "C:\\jaiganesha\\avaragemonthlyincome_p.parquet"
result1_p.to_parquet(output_path1,index=False)
print(f"the output saved successfully to the path{output_path1}\n")
output_path2= "C:\\jaiganesha\\avaragemonthlyincome_j.json"
result1_p.to_json(output_path2,index=False)
print(f"the output saved successfully to the path{output_path2}\n")


Query 1: Average Monthly Income by Department and Education Field
+--------------------+----------------+------------------+
|          Department|  EducationField|avg_monthly_income|
+--------------------+----------------+------------------+
|     Human Resources| Human Resources| 9138.066666666668|
|     Human Resources|           Other|            7988.0|
|               Sales|   Life Sciences| 7409.663366336634|
|               Sales|       Marketing| 7348.131147540984|
|     Human Resources|         Medical| 6909.888888888889|
|     Human Resources|   Life Sciences| 6763.333333333333|
|Research & Develo...|         Medical| 6698.923664122138|
|               Sales|         Medical| 6462.152542372882|
|Research & Develo...|           Other| 6371.744680851064|
|Research & Develo...|Technical Degree| 6277.424242424242|
|Research & Develo...|   Life Sciences| 6193.613496932516|
|               Sales|Technical Degree| 5864.576923076923|
|               Sales|           Other|          

In [31]:
query_2 = """
SELECT 
    JobRole, 
    AVG(JobSatisfaction) AS avg_job_satisfaction 
FROM output_directory 
GROUP BY JobRole 
ORDER BY avg_job_satisfaction DESC 
LIMIT 5
"""
result_2 = spark.sql(query_2)
print("Query 2: Top 5 Job Roles with Highest Satisfaction")
result_2.show()

result2_p=result_2.toPandas()
output_path="C:\\jaiganesha\\satisfactionjobavg.csv"
result2_p.to_csv(output_path,index=False)
print(f"output saved to\t{output_path}\n")
output_path1="C:\\jaiganesha\\satisfactionjobavg_p.parquet"
result2_p.to_parquet(output_path1,index=False)
print(f"output saved to\t{output_path1}\n")
output_path2= "C:\\jaiganesha\\satisfactionjobavg_j.json"
result2_p.to_json(output_path2,index=False)
print(f"the output saved successfully to the path{output_path2}\n")

Query 2: Top 5 Job Roles with Highest Satisfaction
+--------------------+--------------------+
|             JobRole|avg_job_satisfaction|
+--------------------+--------------------+
|Healthcare Repres...|  2.8295454545454546|
|Laboratory Techni...|  2.8138297872340425|
|  Research Scientist|  2.7803738317757007|
|Sales Representative|  2.7586206896551726|
|     Sales Executive|  2.7543103448275863|
+--------------------+--------------------+

output saved to	C:\jaiganesha\satisfactionjobavg.csv

output saved to	C:\jaiganesha\satisfactionjobavg_p.parquet

the output saved successfully to the pathC:\jaiganesha\satisfactionjobavg_j.json



In [32]:
query_3 = """
SELECT 
    JobLevel, 
    Gender, 
    COUNT(*) AS count 
FROM output_directory 
GROUP BY JobLevel, Gender 
ORDER BY JobLevel, count DESC
"""
result_3 = spark.sql(query_3)
print("Query 3: Gender Distribution Across Job Levels")
result_3.show()

result3_p=result_3.toPandas()
output_path="C:\\jaiganesha\\joblevel.csv"
result3_p.to_csv(output_path,index=False)
print(f"the output saved to\t{output_path}\n")
output_path1="C:\\jaiganesha\\joblevel_p.parquet"
result3_p.to_parquet(output_path1,index=False)
print(f"the output saved to\t{output_path1}\n")
output_path2= "C:\\jaiganesha\\avaragemonthlyincome_p.parquet"
result1_p.to_parquet(output_path1,index=False)
print(f"the output saved successfully to the path{output_path2}\n")


Query 3: Gender Distribution Across Job Levels
+--------+------+------+
|JobLevel|Gender| count|
+--------+------+------+
|       1|  Male|153513|
|       1|Female| 90119|
|       2|  Male|101552|
|       2|Female| 84132|
|       3|  Male| 49708|
|       3|Female| 39621|
|       4|  Male| 24195|
|       4|Female| 20346|
|       5|  Male| 19654|
|       5|Female| 11766|
+--------+------+------+

the output saved to	C:\jaiganesha\joblevel.csv

the output saved to	C:\jaiganesha\joblevel_p.parquet

the output saved successfully to the pathC:\jaiganesha\avaragemonthlyincome_p.parquet



In [33]:
query_4 = """
SELECT 
    OverTime, 
    AVG(PerformanceRating) AS avg_performance_rating 
FROM output_directory 
GROUP BY OverTime 
ORDER BY avg_performance_rating DESC
"""
result_4 = spark.sql(query_4)
print("Query 4: Relationship Between OverTime and Performance Rating")
result_4.show()
result4_p=result_4.toPandas()
output_path="C:\\jaiganesha\\performance.csv"
result4_p.to_csv(output_path,index=False)
print(f"output saved to\t{output_path}\n")
output_path1="C:\\jaiganesha\\performance_p.parquet"
result4_p.to_csv(output_path1,index=False)
print(f"output saved to\t{output_path1}\n")
output_path2="C:\\jaiganesha\\performance_j.json"
result4_p.to_csv(output_path2,index=False)
print(f"output saved to\t{output_path2}\n")



Query 4: Relationship Between OverTime and Performance Rating
+--------+----------------------+
|OverTime|avg_performance_rating|
+--------+----------------------+
|      No|    3.1585016947216085|
|     Yes|    3.1505805889567737|
+--------+----------------------+

output saved to	C:\jaiganesha\performance.csv

output saved to	C:\jaiganesha\performance_p.parquet

output saved to	C:\jaiganesha\performance_j.json



In [34]:
query_5 = """
SELECT 
    TotalWorkingYears, 
    YearsAtCompany, 
    COUNT(*) AS employee_count 
FROM output_directory 
GROUP BY TotalWorkingYears, YearsAtCompany 
ORDER BY employee_count DESC
"""
result_5 = spark.sql(query_5)
print("Query 5: Attrition Risk Indicators")
result_5.show()
result5_p=result_5.toPandas()
output_path="C:\\jaiganesha\\riskindicator.csv"
result5_p.to_csv(output_path,index=False)
print(f"output saved to path\t{output_path}\n")
output_path1="C:\\jaiganesha\\riskindicator_p.parquet"
result5_p.to_csv(output_path1,index=False)
print(f"output saved to path\t{output_path}\n")
output_path2="C:\\jaiganesha\\riskindicator_j.json"
result5_p.to_csv(output_path2,index=False)
print(f"output saved to path\t{output_path2}\n")




Query 5: Attrition Risk Indicators
+-----------------+--------------+--------------+
|TotalWorkingYears|YearsAtCompany|employee_count|
+-----------------+--------------+--------------+
|               10|            10|         32737|
|                1|             1|         31906|
|                6|             5|         17387|
|                6|             6|         13487|
|                5|             5|         13361|
|                2|             2|         13168|
|                8|             8|         12085|
|                9|             9|         11984|
|               10|             9|         11027|
|                5|             4|          9306|
|               10|             8|          8987|
|                4|             4|          8668|
|                4|             3|          8618|
|                5|             3|          8349|
|                3|             3|          7459|
|                6|             2|          6628|
|              

In [35]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [36]:
output_path = "C:\\jaiganesha\\output_p.parquet"  
save_data(joined_data_panda, output_path, file_format='parquet', mode='overwrite')

Data saved successfully to C:\jaiganesha\output_p.parquet


In [37]:
import pendulum
print(pendulum.timezone("UTC"))

Timezone('UTC')


In [38]:
def run_command_with_daemon_option(cmd, daemon, pid):
    import subprocess
    subprocess.Popen(cmd)

In [39]:
pip install luigi

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [40]:
import luigi
import time


class FetchDataTask(luigi.Task):
    output_file = luigi.Parameter()

    def run(self):
        print(f"Fetching data and writing to {self.output_file}")
        
        with open(self.output_file, 'w') as f:
            f.write("Simulated data\n")
        time.sleep(2)

class ProcessDataTask(luigi.Task):
    input_file = luigi.Parameter()
    output_file = luigi.Parameter()

    def requires(self):
        
        return FetchDataTask(output_file=self.input_file)

    def run(self):
        print(f"Processing data from {self.input_file} and saving to {self.output_file}")
        with open(self.input_file, 'r') as f:
            data = f.read()

        # Simulating data processing
        processed_data = data.upper()  # Convert text to uppercase
        
        with open(self.output_file, 'w') as f:
            f.write(processed_data)
        time.sleep(2)

# Task C: Store the processed data
class StoreDataTask(luigi.Task):
    input_file = luigi.Parameter()

    def requires(self):
        # This task depends on ProcessDataTask
        return ProcessDataTask(input_file='raw_data.txt', output_file='processed_data.txt')

    def run(self):
        print(f"Storing processed data from {self.input_file}")
        with open(self.input_file, 'r') as f:
            data = f.read()

        # Simulating storing the data
        with open('final_data.txt', 'w') as f:
            f.write(data)
        time.sleep(2)

2025-01-07 16:30:00,667 - INFO - Loaded []


#  Run the pipeline
To execute the pipeline, run the following command in your terminal:

bash
Copy code
python pipeline.py StoreDataTask --input-file=raw_data.txt

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when
import logging
import smtplib
from email.message import EmailMessage

# Logging Configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("SparkJobMonitor")

# Email Alert Function
def send_email_alert(subject, body):
    sender_email = "akshatapandit7@gmail.com"
    receiver_email = "akshata.codes2332@gmail.com"
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_password = "your_email_password"

    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg.set_content(body)

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, smtp_password)
            server.send_message(msg)
        logger.info("Email alert sent successfully.")
    except Exception as e:
        logger.error(f"Failed to send email alert: {e}")

# Spark Job Execution and Monitoring
def run_spark_job():
    spark = SparkSession.builder \
        .appName("AlternativeSparkMonitoring") \
        .getOrCreate()

    try:
        
        data_path = "C:\\excels\\train.csv"
        df = spark.read.csv(data_path, header=True)

        total_rows = df.count()
        null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()

        logger.info(f"Total rows processed: {total_rows}")
        logger.info(f"Null counts: {null_counts}")

        # Alert if row count is below a threshold
        if total_rows < 1000:
            alert_message = f"ALERT: Row count below threshold! Processed rows: {total_rows}"
            logger.warning(alert_message)
            send_email_alert("Row Count Alert", alert_message)

        # Alert if null values exceed a threshold
        for column, null_count in zip(df.columns, null_counts[0]):
            if null_count > 0:
                alert_message = f"ALERT: Null values detected in column '{column}': {null_count}"
                logger.warning(alert_message)
                send_email_alert("Data Quality Alert", alert_message)

    except Exception as e:
        logger.error(f"Spark job encountered an error: {e}")
        send_email_alert("Job Failure Alert", f"Spark job failed with error: {e}")

    finally:
        spark.stop()
        logger.info("Spark session stopped.")


if __name__ == "__main__":
    run_spark_job()


2025-01-07 16:30:02,531 - INFO - Total rows processed: 1058
2025-01-07 16:30:02,534 - INFO - Null counts: [Row(Age=0, Attrition=0, BusinessTravel=0, DailyRate=0, Department=0, DistanceFromHome=0, Education=0, EducationField=0, EmployeeCount=0, EmployeeNumber=0, EnvironmentSatisfaction=0, Gender=0, HourlyRate=0, JobInvolvement=0, JobLevel=0, JobRole=0, JobSatisfaction=0, MaritalStatus=0, MonthlyIncome=0, MonthlyRate=0, NumCompaniesWorked=0, Over18=0, OverTime=0, PercentSalaryHike=0, PerformanceRating=0, RelationshipSatisfaction=0, StandardHours=0, StockOptionLevel=0, TotalWorkingYears=0, TrainingTimesLastYear=0, WorkLifeBalance=0, YearsAtCompany=0, YearsInCurrentRole=0, YearsSinceLastPromotion=0, YearsWithCurrManager=0)]
2025-01-07 16:30:03,834 - INFO - Spark session stopped.
